# Heating Mesh Tally on CAD geometry made from Shapes

This constructs a reactor geometry from 3 Shape objects each made from points.

The Shapes made include a breeder blanket, PF coil and a central column shield.

2D and 3D Meshes tally are then simulated to show nuclear heating, flux and tritium_production across the model.

This makes a 3D geometry and material for the centre column

In [ ]:
import paramak

center_column = paramak.RotateMixedShape(
    points=[
        (50, 600, 'straight'),
        (150, 600, 'spline'),
        (100, 0, 'spline'),
        (150, -600, 'straight'),
        (50, -600, 'straight')
    ],
)

This makes a 3D geometry and material for breeder blanket. The azimuth_placement_angle argument is used to repeat the geometry around the Z axis at specified angles.

In [ ]:
blanket = paramak.RotateSplineShape(
    points=[
        (600, 0),
        (600, -20),
        (500, -300),
        (400, -300),
        (400, 0),
        (400, 300),
        (500, 300),
        (600, 20)
    ],
    rotation_angle=40,
    azimuth_placement_angle=[0, 45, 90, 135, 180, 225, 270, 315],
)

This makes a reactor object from the three components

In [ ]:
my_reactor = paramak.Reactor([blanket, center_column])
my_reactor.export_stl(['plasma.stl', 'center_column.stl'])
my_reactor.show()

This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified. There are a few different methods of converting a CAD model into a neutronics model. Each method has pros and cons.

The rector can also be converted from stl or stp to h5m format which is the 3D geometry format used by DAGMC to represent neutronics geometry.

stl-to-h5m can convert stl files to a h5m geometry
https://github.com/fusion-energy/stl_to_h5m

cad-to-h5m can convert stp files to h5m geometry
https://github.com/fusion-energy/cad_to_h5m

In [ ]:
from stl_to_h5m import stl_to_h5m

stl_to_h5m(
    files_with_tags=[
        ('plasma.stl', 'plasma'),
        ('center_column.stl', 'center_column')
    ],
    h5m_filename='dagmc.h5m',
)

This next section combines the geometry with the materials and specifies a few mesh tallies

In [ ]:

# makes use of the previously created neutronics geometry (h5m file) and assigns
# actual materials to the material tags. Sets simulation intensity and specifies
# the neutronics results to record (know as tallies).

import openmc
import openmc_dagmc_wrapper as odw
import openmc_plasma_source as ops
import neutronics_material_maker as nmm

# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'plasma': nmm.Material.from_library(name='DT_plasma'),
    'center_column': nmm.Material.from_library(name='Li4SiO4'),
}

geometry = odw.Geometry(h5m_filename='dagmc.h5m')

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)

tally1 = odw.MeshTally3D(
    mesh_resolution=(100, 100, 100),
    bounding_box=[(500,500,500), (-500,-500,-500)],
    tally_type="(n,Xa)",
)

tallies = openmc.Tallies([tally1])

settings = odw.FusionSettings()
settings.batches = 4
settings.particles = 1000
# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package
settings.source = ops.FusionRingSource(fuel="DT", radius=350)


my_model = openmc.Model(
    materials=materials, geometry=geometry, settings=settings, tallies=tallies
)
statepoint_file = my_model.run()

Work in progress

method of extracting vtk file is in progress https://github.com/fusion-energy/regular_mesh_plotter/issues/6

The next section produces download links for:

- vtk files that contain the 3D mesh results (open with Paraview)
- png images that show the resuls of the 2D mesh tally

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_on_3D_mesh.vtk'))
display(FileLink('flux_on_3D_mesh.vtk'))
display(FileLink('tritium_production_on_3D_mesh.vtk'))
display(FileLink('flux_on_2D_mesh_xy.png'))
display(FileLink('flux_on_2D_mesh_xz.png'))
display(FileLink('flux_on_2D_mesh_yz.png'))
display(FileLink('heating_on_2D_mesh_xy.png'))
display(FileLink('heating_on_2D_mesh_xz.png'))
display(FileLink('heating_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))
display(FileLink('tritium_production_on_2D_mesh_xz.png'))
display(FileLink('tritium_production_on_2D_mesh_yz.png'))